In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pennylane as qml
from matplotlib import pyplot as plt
from pennylane import numpy as np
import scipy
import networkx as nx
import copy

In [10]:
weighted_graph_adj_matrix = np.array([
    [0., 1., 2., 1.],
    [1., 0., 3., 2.],
    [2., 3., 0., 1.],
    [1., 2., 1., 0.]
])

In [11]:
from qgscnn import CouplingGate


In [29]:
def compute_L_matrix(adj_matrix):
    vertex_sums = np.sum(adj_matrix, axis=0)
    out = np.zeros(adj_matrix.shape)
    for j in range(len(adj_matrix)):
        for k in range(len(adj_matrix)):
            if j != k:
                out[j][k] = -1. * adj_matrix[j][k]
            else:
                out[j][k] = vertex_sums[j] - adj_matrix[j][k]
                
    return out
        

In [42]:
def target_hamiltonian(L_matrix):
    out = np.zeros((4,4))
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            outer_prod_i = np.array([
                [0.,0.],
                [0., 1.]
            ])
            
            outer_prod_j = np.array([
                [0., 0.],
                [0., 1.]
            ])
            
            out += L_matrix[i][j] * np.kron(outer_prod_i, outer_prod_j)
    return out + np.identity(len(L_matrix))

In [43]:
def target_to_qml_hamiltonian(target):
    """
    Helper function to turn a matrix representing the target Hamiltonian into a Pennylane Hamiltonian object
    """
    pauli_coeffs, obs_list = qml.utils.decompose_hamiltonian(target)
    return qml.Hamiltonian(pauli_coeffs, obs_list)
    

In [44]:
def coupling_sublayer(L_matrix, weights, wires):
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            outer_prod_i = np.array([
                [1.-np.exp(-1j*weights[j]/2),0.],
                [0., 1.-np.exp(-1j*weights[j]/2)]
            ])

            outer_prod_j = np.array([
                [1.-np.exp(-1j*weights[j]/2), 0.],
                [0., 1.+np.exp(-1j*weights[j]/2)]
            ])

            gate_matrix = L_matrix[i][j] * np.kron(outer_prod_i, outer_prod_j)

            CouplingGate(gate_matrix, weights[i], weights[j], wires=list(set([i,j])))
    

In [45]:
def qgscnn_layer(weights, wires, L_matrix):
    # add the coupling gates
    #coupling_sublayer(L_matrix, weights[0], wires)
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i != j:
                if L_matrix[i][j] != 0.:
                    qml.CNOT(wires=[i,j])

    # add X rotations for the kinetic hamiltonian
    for i in range(len(L_matrix)):
        qml.RX(weights[0][i], wires=i)
    

In [50]:
def quantum_clustering(adj_matrix):

    # the quantum device
    dev = qml.device("default.qubit", wires=len(adj_matrix))
    
    # compute the L matrix
    L_matrix = compute_L_matrix(adj_matrix)
    
    # helps the cost fn if we declare how many vertices graph has
    num_vertices = len(adj_matrix)
    
    # compute the target Hamiltonian
    target = target_hamiltonian(L_matrix)
    
    target_H = target_to_qml_hamiltonian(target)
    
    # model set up
    optimiser = qml.AdamOptimizer(stepsize=0.5)
    steps = 100
    num_layers = 5
    
    def qgscnn(weights, wires, L_matrix):
        for i in range(num_layers):
            qgscnn_layer(weights[i], wires, L_matrix)
    
    # Defines the new QNode
    qnode = qml.QNode(qgscnn, dev)
    
#     def clustering_cost_fn(params):
#         state = qnode(params, wires=range(num_vertices), L_matrix=L_matrix)
#         print(state)
#         print(target)
#         loss = np.vdot(state, (target @ state))
#         print(loss)
#         return loss

    def ansatz(params, **kwargs):
        return qgscnn(params, wires=range(len(adj_matrix)), L_matrix=L_matrix)
        

    cost_fn = qml.ExpvalCost(ansatz, target_H, dev)
    
    # to make life easier, kinetic weights are weights[layer][1], and coupling weights are weights[layer][0]
    weights = np.random.randint(-20, 20, size=(num_layers, 1, adj_matrix.shape[1]))/50
    init = copy.copy(weights)

    # Executes the optimization method
    iterations = 0

    for i in range(0, steps):
        print(f"Iteration {i}")
        weights = optimiser.step(cost_fn, weights)

    return weights, init

In [51]:
quantum_clustering(weighted_graph_adj_matrix)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

(tensor([[[ 3.39999908e-01,  3.99997916e-02, -2.00001028e-02,
            3.19999914e-01]],
 
         [[ 3.79999931e-01, -3.60000076e-01, -6.00002152e-02,
           -1.80000108e-01]],
 
         [[ 1.39999885e-01, -4.00000912e-02, -3.80000093e-01,
           -2.00000204e-01]],
 
         [[ 1.99998869e-02,  1.59999929e-01,  2.39999926e-01,
            2.39999913e-01]],
 
         [[-2.20000240e-01,  3.19999888e-01,  1.99999948e-01,
           -1.60000087e-01]],
 
         [[-1.40000103e-01, -1.80000172e-01, -3.40000107e-01,
            1.59999955e-01]],
 
         [[-1.60000030e-01, -2.60000106e-01, -3.40000160e-01,
            1.59999888e-01]],
 
         [[-2.40000111e-01,  1.59999944e-01, -3.60000103e-01,
           -3.40000131e-01]],
 
         [[ 3.39999894e-01, -3.80000051e-01, -3.40000048e-01,
            2.19999936e-01]],
 
         [[-2.20000081e-01,  2.59999882e-01,  3.19999923e-01,
            2.39999963e-01]],
 
         [[ 3.79999943e-01, -4.00000039e-01,  2.79999881e-01